In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [ ]:
filename_grokKey = "..\\GroqAPIKey.txt"

In [6]:
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### RunnablePassthrough, RunnableLambda and RunnableParallel

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableParallel
from langchain_groq import ChatGroq

In [13]:
classifier_template = PromptTemplate(
    input_variables=["question","answer"],
    template="""You are given a question and the user's response to that question. Classifiy the response as either `Correct` , or  `Incorrect`.
    Do not respond with more than one word. 
    Question - {question}
    User's Answer - {answer}
    Classification:"""
)

In [18]:
#gemini_llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
lamma_llm = ChatGroq(model="llama-3.1-8b-instant")

In [19]:
output_parser = StrOutputParser()

In [20]:
classifier_chain = classifier_template | lamma_llm | output_parser

In [21]:
classifier_chain.invoke({"question":"what are penguins","answer":"Penguins are sea animals"})

'Incorrect'

In [22]:
classifier_chain.invoke({"question":"what are penguins","answer":"Penguins are birds"})   

'Correct'

In [23]:
correct_template = PromptTemplate(
    input_variables=["question"],
    template=""" The user was asked the following question and user answered it correctly. 
    Now ask a more difficult question on the same topic to the user.
    Question: {question}
    New Question:"""
)

correct_chain = correct_template | lamma_llm | output_parser

incorrect_template = PromptTemplate(
    input_variables=["question"],
    template=""" The user was asked the following question and user answered it incorrectly. 
    Give the correct answer  and explain it to the user.
    Question: {question}
    Correct Answer:
    Explanation:"""
)

incorrect_chain = incorrect_template | lamma_llm | output_parser


In [24]:
def route(info):
    if info["result"].lower() == "correct":
        return correct_chain
    elif info["result"].lower() == "incorrect":
        return incorrect_chain
    else:
        return "Format is not correct"

In [25]:
final_chain = RunnableParallel({"result":classifier_chain,"question":lambda x: x["question"],"answer": lambda x: x["answer"]}) | \
              RunnableParallel({"response":RunnableLambda(route),"input":RunnablePassthrough()})

In [26]:
final_chain.invoke({"question":"what are penguins","answer":"Penguins are birds"})

{'response': "You answered correctly that penguins are a type of flightless bird. \n\nHere's a more challenging question on the same topic:\n\nWhat is the main difference between the Emperor Penguin and the King Penguin in terms of their breeding habits, and how do these differences affect their chick-rearing strategies?",
 'input': {'result': 'Correct',
  'question': 'what are penguins',
  'answer': 'Penguins are birds'}}

In [27]:
final_chain.invoke({"question":"what are penguins","answer":"Penguins are sea animals"})

{'response': "Correct Answer: Penguins are flightless birds that belong to the family Spheniscidae.\n\nExplanation: Penguins are a group of aquatic birds that are found primarily in the Southern Hemisphere. They are characterized by their distinctive black and white feathers, which help them blend in with the sea and sky, making it easier for them to hunt and avoid predators. Penguins are flightless birds, meaning they are not able to fly like other birds. This is due to their unique body shape, which includes a streamlined body, short wings, and powerful flippers.\n\nPenguins are excellent swimmers, using their flippers to propel themselves through the water in search of food. They are carnivorous, feeding on a variety of prey such as fish, krill, and squid. Penguins are highly social animals, often living in large colonies and engaging in complex behaviors such as mating rituals and parental care.\n\nThere are 18 different species of penguins, ranging in size from the little blue pen

In [30]:
final_chain.invoke({"question":"what are SQL joins","answer":"Joins are used to join two queries"})

{'response': "**Correct Answer:** SQL joins are used to combine rows from two or more tables based on a related column between them.\n\n**Explanation:**\n\nImagine you have two tables, `orders` and `customers`, and you want to see a list of customers along with their corresponding order information. However, the `orders` table contains a foreign key, `customer_id`, that references the `id` column in the `customers` table. This is where SQL joins come in.\n\nThere are three main types of SQL joins:\n\n1. **INNER JOIN**: Returns only the rows that have a match in both tables. If a row in the `orders` table doesn't have a matching `customer_id` in the `customers` table, it won't be included in the result.\n2. **LEFT JOIN** (or **LEFT OUTER JOIN**): Returns all the rows from the left table (`orders`) and the matching rows from the right table (`customers`). If there's no match, the result will contain NULL values for the columns from the right table.\n3. **RIGHT JOIN** (or **RIGHT OUTER JO

In [31]:
final_chain.invoke({"question":"what are SQL joins","answer":"SQL joins are used to combine rows from two or more tables based on a related column between them."})

{'response': "Now that you know what SQL joins are, let's try a more challenging question:\n\nWhat are the different types of SQL joins, and how do they differ in terms of the data they return? Please list at least 3 types of SQL joins and describe a scenario where you would use each one.",
 'input': {'result': 'Correct',
  'question': 'what are SQL joins',
  'answer': 'SQL joins are used to combine rows from two or more tables based on a related column between them.'}}